In [3]:
import os
from pathlib import Path

import pandas as pd
import numpy as np
import pandas_datareader.data as web


In [130]:
df_prices = pd.read_parquet('./dataset/0000.parquet')
df_prices['date'] = pd.to_datetime(df_prices['date'])
print("Min date of dataset: ", df_prices.date.min())
print("Max date of dataset: ", df_prices.date.max())
df_prices[df_prices['ticker']=='BTC'].tail()

Min date of dataset:  2010-07-17 00:00:00
Max date of dataset:  2023-08-25 00:00:00


,ticker,date,open,high,low,close
29771,BTC,2023-08-21,26178.7,26246.8,25812.0,26192.4
29772,BTC,2023-08-22,26192.1,26192.3,25294.2,25660.3
29773,BTC,2023-08-23,25660.6,26805.7,25634.5,26315.0
29774,BTC,2023-08-24,26312.0,26566.7,25857.2,26081.7
29775,BTC,2023-08-25,26081.7,26296.3,25765.6,26067.5


In [8]:
df_tickers = pd.read_csv('./dataset/tickers.csv', index_col = 0)
print(df_tickers.shape)
df_tickers.T.head(10)

#df_tickers

(30, 1)


#,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
ticker,BTC,ETH,BNB,SOL,XRP,ADA,AVAX,LINK,DOGE,TRX,...,ATOM,IMX,OP,NEAR,XLM,INJ,OKB,FIL,HBAR,XMR


In [104]:
import requests
import time
from datetime import date

def get_coin_history(fsym, tsym, limit, toTs, api_key):    
    tm = int(time.mktime(toTs.timetuple()))
    url = (f"https://min-api.cryptocompare.com/data/v2/histoday?fsym={fsym}&tsym={tsym}&limit={limit}&toTs={tm}&api_key={api_key}")    
    header = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36","X-Requested-With": "XMLHttpRequest"}
    data = requests.get(url, headers=header).json()    
    df = pd.DataFrame(data['Data']['Data'])
    df['date'] = df.time.apply(lambda x: datetime.utcfromtimestamp(x))
    df['ticker'] = fsym
    df = df[['ticker', 'date', 'open', 'high', 'low', 'close' ]]

    return df

In [105]:
start_date = date(2023, 8, 26)
end_date = date(2024, 2, 15) # end date not included in results
limit = (end_date - start_date).days - 1
print(limit)
df = get_coin_history('BTC', 'USD', limit, end_date, 'e03fd0d70508d6ad46fedbbc85bfb03aedb2d3120dcd9760b0ca4c88abcc2e66')
df

172


,ticker,date,open,high,low,close
0,BTC,2023-08-26,26051.29,26109.19,25979.58,26010.35
1,BTC,2023-08-27,26010.35,26168.79,25966.75,26091.32
2,BTC,2023-08-28,26091.32,26233.53,25861.84,26106.00
3,BTC,2023-08-29,26106.00,28154.08,25911.89,27724.41
4,BTC,2023-08-30,27724.41,27766.41,27036.49,27305.41
...,...,...,...,...,...,...
168,BTC,2024-02-10,47154.23,48166.14,46894.61,47768.12
169,BTC,2024-02-11,47768.12,48597.46,47592.04,48311.16
170,BTC,2024-02-12,48311.16,50339.20,47718.59,49942.97
171,BTC,2024-02-13,49942.97,50392.92,48336.83,49733.11


In [110]:
from datetime import date
start_date = date(2023, 8, 26)
end_date = date(2024, 2, 15) # end date not included in results
limit = (end_date - start_date).days - 1
print(limit)

df = pd.DataFrame()
#for ticker in ['BTC', 'ETH']:    
for ticker in df_tickers['ticker']:
    df = pd.concat([df, get_coin_history(ticker, 'USD', limit, end_date, 'e03fd0d70508d6ad46fedbbc85bfb03aedb2d3120dcd9760b0ca4c88abcc2e66')])
print(df.shape)
df

172
(5190, 6)


,ticker,date,open,high,low,close
0,BTC,2023-08-26,26051.29,26109.19,25979.58,26010.35
1,BTC,2023-08-27,26010.35,26168.79,25966.75,26091.32
2,BTC,2023-08-28,26091.32,26233.53,25861.84,26106.00
3,BTC,2023-08-29,26106.00,28154.08,25911.89,27724.41
4,BTC,2023-08-30,27724.41,27766.41,27036.49,27305.41
...,...,...,...,...,...,...
168,XMR,2024-02-10,121.71,124.34,116.46,119.23
169,XMR,2024-02-11,119.84,123.46,115.44,120.76
170,XMR,2024-02-12,120.63,128.60,119.76,126.36
171,XMR,2024-02-13,126.45,128.82,122.37,127.87


In [111]:
df.pivot(index='date', columns='ticker', values=['close'])

close                                                           \
ticker         ADA    ATOM   AVAX     BCH     BNB       BTC     DOGE    DOT   
date                                                                          
2023-08-26  0.2610   7.153  10.09  190.05  216.41  26010.35  0.06287  4.505   
2023-08-27  0.2633   7.179  10.21  197.73  218.28  26091.32  0.06315  4.490   
2023-08-28  0.2675   7.262  10.37  190.47  218.85  26106.00  0.06342  4.607   
2023-08-29  0.2727   7.446  10.81  225.16  226.90  27724.41  0.06639  4.675   
2023-08-30  0.2655   7.223  10.40  215.99  223.56  27305.41  0.06588  4.535   
...            ...     ...    ...     ...     ...       ...      ...    ...   
2024-02-10  0.5517   9.970  40.23  247.79  323.29  47768.12  0.08145  7.202   
2024-02-11  0.5418   9.744  39.73  277.64  320.88  48311.16  0.08125  7.101   
2024-02-12  0.5596   9.917  41.00  280.66  328.03  49942.97  0.08226  7.344   
2024-02-13  0.5452  10.290  39.75  270.53  324.96  49733.11  0.08110  7.358   
2024-02-14  0.5775  10.250  42.26  281.06  334.24  51844.19  0.08556  7.656   

                            ...                                         \
ticker        ETC      ETH  ...    OKB     OP      SHIB     SOL    TON   
date                        ...                                          
2023-08-26  15.86  1646.30  ...  42.94  1.481  0.000008   20.28  1.274   
2023-08-27  16.02  1657.60  ...  43.10  1.453  0.000008   20.79  2.399   
2023-08-28  15.89  1652.22  ...  43.02  1.422  0.000008   20.55  1.276   
2023-08-29  16.87  1729.38  ...  43.92  1.495  0.000008   21.79  1.283   
2023-08-30  16.02  1705.51  ...  43.85  1.481  0.000008   20.82  1.246   
...           ...      ...  ...    ...    ...       ...     ...    ...   
2024-02-10  25.87  2500.87  ...  49.97  3.366  0.000009  109.07  1.927   
2024-02-11  25.88  2507.91  ...  49.95  3.536  0.000009  107.48  2.000   
2024-02-12  26.94  2660.56  ...  49.70  3.816  0.000010  111.73  2.049   
2024-02-13  26.22  2641.60  ...  49.50  3.731  0.000009  112.57  1.980   
2024-02-14  26.88  2777.51  ...  50.20  3.824  0.000010  117.11  2.022   

                                                    
ticker          TRX    UNI     XLM     XMR     XRP  
date                                                
2023-08-26  0.07738  4.599  0.1209  143.00  0.5232  
2023-08-27  0.07746  4.683  0.1209  145.35  0.5238  
2023-08-28  0.07633  4.668  0.1199  145.18  0.5230  
2023-08-29  0.07732  4.799  0.1244  146.86  0.5398  
2023-08-30  0.07564  4.640  0.1204  142.07  0.5283  
...             ...    ...     ...     ...     ...  
2024-02-10  0.12450  6.630  0.1117  119.23  0.5241  
2024-02-11  0.12420  6.680  0.1113  120.76  0.5263  
2024-02-12  0.12500  6.839  0.1132  126.36  0.5317  
2024-02-13  0.12690  6.681  0.1110  127.87  0.5247  
2024-02-14  0.12960  6.894  0.1145  128.16  0.5383  

[173 rows x 30 columns]

In [112]:
df.to_parquet('./dataset/0001.parquet')

In [113]:
df_prices_1 = pd.read_parquet('./dataset/0001.parquet')
print("Min date of dataset: ", df_prices_1.date.min())
print("Max date of dataset: ", df_prices_1.date.max())
df_prices_1[df_prices_1['ticker']=='BTC'].tail()

Min date of dataset:  2023-08-26 00:00:00
Max date of dataset:  2024-02-14 00:00:00


,ticker,date,open,high,low,close
168,BTC,2024-02-10,47154.23,48166.14,46894.61,47768.12
169,BTC,2024-02-11,47768.12,48597.46,47592.04,48311.16
170,BTC,2024-02-12,48311.16,50339.20,47718.59,49942.97
171,BTC,2024-02-13,49942.97,50392.92,48336.83,49733.11
172,BTC,2024-02-14,49733.11,52085.66,49272.58,51844.19


In [132]:
df_all_prices = pd.concat([df_prices, df_prices_1])
print(df_all_prices.shape)
df_all_prices

(209207, 6)


,ticker,date,open,high,low,close
0,1INCH,2021-01-08,1.300,1.357,1.1430,1.200
1,1INCH,2021-01-09,1.200,1.340,1.1140,1.244
2,1INCH,2021-01-10,1.244,1.547,1.1000,1.224
3,1INCH,2021-01-11,1.224,1.224,0.9543,1.127
4,1INCH,2021-01-12,1.127,1.210,1.0490,1.117
...,...,...,...,...,...,...
168,XMR,2024-02-10,121.710,124.340,116.4600,119.230
169,XMR,2024-02-11,119.840,123.460,115.4400,120.760
170,XMR,2024-02-12,120.630,128.600,119.7600,126.360
171,XMR,2024-02-13,126.450,128.820,122.3700,127.870


In [139]:
df_selected_prices = df_all_prices[df_prices['ticker'].isin(df_tickers['ticker'])]
df_pivoted = df_selected_prices.pivot(index='date', columns='ticker', values=['close'])
df_pivoted = df_pivoted.dropna()
print(df_pivoted.shape)
df_pivoted

(463, 28)


C:\Users\lukin\AppData\Local\Temp\ipykernel_10948\4051034494.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_selected_prices = df_all_prices[df_prices['ticker'].isin(df_tickers['ticker'])]


close                                                           \
ticker           ADA      ATOM      AVAX      BCH      BNB      BTC      DOGE   
date                                                                            
2022-05-14  0.528559  10.67400  32.74080  230.177  288.795  29661.4  0.087977   
2022-05-15  0.583241  12.04430  36.34270  212.623  308.598  31029.6  0.091681   
2022-05-16  0.569958  11.26530  33.74610  200.339  300.309  30135.2  0.088285   
2022-05-17  0.577049  11.47510  34.03570  206.831  305.003  30512.1  0.090283   
2022-05-18  0.523818  10.32660  30.62010  191.650  293.618  29152.1  0.085189   
...              ...       ...       ...      ...      ...      ...       ...   
2023-08-21  0.264870   7.58286  10.39850  186.972  211.268  26192.4  0.062650   
2023-08-22  0.253311   7.25053   9.89644  180.939  206.728  25660.3  0.061370   
2023-08-23  0.267559   7.36453  10.26810  193.161  215.459  26315.0  0.063439   
2023-08-24  0.265055   7.16035  10.06600  191.502  217.089  26081.7  0.062649   
2023-08-25  0.260557   7.22502   9.99450  192.894  217.980  26067.5  0.062876   

                                        ...                              \
ticker           DOT      ETC      ETH  ...     MATIC     NEAR      OKB   
date                                    ...                               
2022-05-14  11.15440  20.9286  2024.99  ...  0.668063  6.51087  12.5405   
2022-05-15  11.57490  21.7817  2132.61  ...  0.721061  7.19463  12.9995   
2022-05-16  10.82590  21.0826  2047.71  ...  0.693098  6.74921  12.4700   
2022-05-17  11.06560  21.5719  2084.22  ...  0.726055  6.55247  12.8513   
2022-05-18   9.78726  20.1038  1958.90  ...  0.643163  6.04513  12.2541   
...              ...      ...      ...  ...       ...      ...      ...   
2023-08-21   4.46813  15.8276  1673.39  ...  0.562170  1.13484  43.3339   
2023-08-22   4.32063  15.1549  1603.45  ...  0.535713  1.15541  42.5755   
2023-08-23   4.46525  16.0669  1664.94  ...  0.561092  1.20067  43.8417   
2023-08-24   4.37907  15.9419  1653.35  ...  0.543136  1.18879  42.8781   
2023-08-25   4.46694  15.9612  1650.48  ...  0.544096  1.19194  42.8364   

                                                                               
ticker          SHIB      SOL       TRX      UNI       XLM      XMR       XRP  
date                                                                           
2022-05-14  0.000012  50.9086  0.071809  5.09286  0.137493  150.474  0.421609  
2022-05-15  0.000013  57.3853  0.071444  5.37301  0.143014  170.678  0.441326  
2022-05-16  0.000012  54.8686  0.069929  5.10336  0.137621  166.184  0.428043  
2022-05-17  0.000013  56.4266  0.072695  5.33306  0.139119  173.075  0.437331  
2022-05-18  0.000012  50.8238  0.071477  5.00349  0.131429  158.094  0.413262  
...              ...      ...       ...      ...       ...      ...       ...  
2023-08-21  0.000008  21.3377  0.075249  4.81838  0.124181  149.478  0.523821  
2023-08-22  0.000008  19.9233  0.074565  4.56625  0.118828  143.751  0.513420  
2023-08-23  0.000008  21.3440  0.077385  4.80919  0.126196  144.071  0.526500  
2023-08-24  0.000008  20.8664  0.076391  4.63434  0.122807  138.329  0.517086  
2023-08-25  0.000008  20.3166  0.077038  4.52337  0.123182  140.021  0.524901  

[463 rows x 28 columns]

### Saving and reading to HDFS

In [4]:
import pyarrow as pa

In [5]:
os.environ['ARROW_LIBHDFS_DIR'] = '/home/student/hadoop/lib/native'
hdfs_interface = pa.hdfs.connect(host='localhost', port=8020, user='student')

C:\Users\lukin\AppData\Local\Temp\ipykernel_18720\203335781.py:2: FutureWarning: pyarrow.hdfs.connect is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  hdfs_interface = pa.hdfs.connect(host='localhost', port=8020, user='student')


FileNotFoundError: [WinError 2] Не удается найти указанный файл

In [ ]:
hdfs_interface.ls('/user/student/')

In [ ]:
cwd = Path('./dataset/')
destination_path = '/user/student/project/dataset/'

for f in cwd.rglob('*.parquet'):
    print(f'uploading {f.name}')
    with open(str(f), 'rb') as f_upl:
        hdfs_interface.upload(destination_path + f.name, f_upl)

hdfs_interface.ls(destination_path)

In [ ]:
table = hdfs_interface.read_parquet('/user/student/project/dataset/0000.parquet')
table_df = table.to_pandas()
table_df.head()